In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Data Mining Field-oriented/consolidated/tokenized_articles_training.tsv', sep='\t')

print(df.columns)
print(df['content'].dtype)

Index(['content', 'tags'], dtype='object')
object


In [ ]:
total_tags = df['tags'].apply(lambda x: len(x.split(','))).sum()

print(f"Tổng số tag trong toàn bộ dữ liệu: {total_tags}")

Tổng số tag trong toàn bộ dữ liệu: 498108


In [ ]:
patterns = {
    'date': r'\b(?:0[1-9]|[12][0-9]|3[01])/(?:0[1-9]|1[0-2])/\d{4}\b|\b(?:0[1-9]|1[0-2])/(?:0[1-9]|[12][0-9]|3[01])/\d{4}\b',
    'special_tags': r'\b[A-Za-z]+\d+\b|\b\d+[A-Za-z]+\b|\b\d+[-_/][A-Za-z\d]+(?:[-_/][A-Za-z\d]+)*\b',
    'long_numbers': r'\b\d{6,}(?:[.,]\d{3})*\b(?![A-Za-z])',
    'isbn_pattern': r'\b(?:\d{9}[\dXx]|\d{13})\b',
    'phone': r'\b(?:\+?\d{1,3}[ -]?)?(?:\d{2,4}[ -]?){2,3}\d{3,4}\b(?!-\d{2}-\d{4})',
    'ip': r'\b(?:\d{1,3}\.){3}\d{1,3}\b|\b(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}\b',
    'percentage': r'\b\d+(?:,\d+)?%\b',
    'score_pattern': r'\b\d{1,2}\s*-\s*\d{1,2}\b',
    'time': r'\b\d{1,2}h(?:\d{2})?\b|\b\d{1,2}:\d{2}\b|\b\d{1,2}\+\d{1,2}\b',
    'comments': r'Ảnh\s*:\s*(?:@\s*)?\w+',
    'mail' : r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b',
    'url' : r'\b(?:https?://|www\.)[^\s/$.?#].[^\s]*\b'
}

def count_pattern_occurrences(pattern, text):
    return len(re.findall(pattern, text))

In [ ]:
import pandas as pd
import re

pattern_counts = {key: 0 for key in patterns}

detailed_counts = []

for pattern_name, pattern in patterns.items():
    tag_counts = df['tags'].apply(lambda x: count_pattern_occurrences(pattern, x))

    for index, count in tag_counts.items():
        if count > 0:
            matched_patterns = re.findall(pattern, df.loc[index, 'content'])
            if matched_patterns:
                matched_patterns = [item if isinstance(item, str) else ''.join(item) for item in matched_patterns]

                matched_patterns_str = ', '.join(matched_patterns)
            else:
                matched_patterns_str = ''

            detailed_counts.append({
                'index': index,
                'matched_patterns': matched_patterns_str,
                'pattern': pattern_name,
                'count': count
            })

    pattern_counts[pattern_name] = tag_counts.sum()

pattern_percentages = {key: (count / total_tags) * 100 for key, count in pattern_counts.items()}

for pattern_name, percentage in pattern_percentages.items():
    count = pattern_counts[pattern_name]
    print(f"Tần suất và phần trăm xuất hiện của {pattern_name}: {count:.2f}, {percentage:.2f}%")

detailed_output_file = 'pattern_occurrences_details_tags.csv'
detailed_df = pd.DataFrame(detailed_counts)
detailed_df.to_csv(detailed_output_file, index=False, encoding='utf-8-sig')

print(f"Chi tiết số lần xuất hiện đã được lưu vào file '{detailed_output_file}'")

Tần suất và phần trăm xuất hiện của date: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của long_numbers: 4.00, 0.00%
Tần suất và phần trăm xuất hiện của isbn_pattern: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của ip: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của percentage: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của time: 1.00, 0.00%
Tần suất và phần trăm xuất hiện của comments: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của mail: 0.00, 0.00%
Tần suất và phần trăm xuất hiện của url: 0.00, 0.00%
Chi tiết số lần xuất hiện đã được lưu vào file 'pattern_occurrences_details_tags.csv'


In [ ]:
import pandas as pd
import re

def tokenize(text):
    return re.findall(r'\b\w+(?:_\w+)*\b', text)


total_tokens = df['content'].apply(lambda x: len(tokenize(x))).sum()

pattern_counts = {key: 0 for key in patterns}

detailed_counts = []

for pattern_name, pattern in patterns.items():
    tag_counts = df['content'].apply(lambda x: count_pattern_occurrences(pattern, x))

    for index, count in tag_counts.items():
        if count > 0:
            matched_patterns = re.findall(pattern, df.loc[index, 'content'])
            if matched_patterns:
                matched_patterns = [item if isinstance(item, str) else ''.join(item) for item in matched_patterns]

                matched_patterns_str = ', '.join(matched_patterns)
            else:
                matched_patterns_str = ''

            detailed_counts.append({
                'index': index,
                'matched_patterns': matched_patterns_str,
                'pattern': pattern_name,
                'count': count
            })

    pattern_counts[pattern_name] = tag_counts.sum()

pattern_percentages = {key: (count / total_tokens) * 100 for key, count in pattern_counts.items()}

for pattern_name, percentage in pattern_percentages.items():
    count = pattern_counts[pattern_name]
    print(f"Tần suất và phần trăm xuất hiện của {pattern_name}: {count:.2f}, {percentage:.2f}%")

detailed_output_file = 'pattern_occurrences_detail_content.csv'
detailed_df = pd.DataFrame(detailed_counts)
detailed_df.to_csv(detailed_output_file, index=False, encoding='utf-8-sig')

print(f"Chi tiết số lần xuất hiện đã được lưu vào file '{detailed_output_file}'")

Tần suất và phần trăm xuất hiện của date: 3896.00, 0.03%
Tần suất và phần trăm xuất hiện của long_numbers: 269.00, 0.00%
Tần suất và phần trăm xuất hiện của isbn_pattern: 68.00, 0.00%
Tần suất và phần trăm xuất hiện của ip: 168.00, 0.00%
Tần suất và phần trăm xuất hiện của percentage: 5.00, 0.00%
Tần suất và phần trăm xuất hiện của time: 4419.00, 0.04%
Tần suất và phần trăm xuất hiện của comments: 466.00, 0.00%
Tần suất và phần trăm xuất hiện của mail: 91.00, 0.00%
Tần suất và phần trăm xuất hiện của url: 295.00, 0.00%
Chi tiết số lần xuất hiện đã được lưu vào file 'pattern_occurrences_detail_content.csv'


In [ ]:
def remove_patterns(text, patterns):
    for pattern in patterns.values():
        text = re.sub(pattern, '', text)
    return text

In [ ]:
df['content'] = df['content'].apply(lambda x: remove_patterns(x, patterns))

df.to_csv('cleaned_noise_1.csv', index=False, encoding='utf-8-sig')

# Test

In [ ]:
import re

def remove_special_num(text):
    pattern_date = (
    r'\b(?:0[1-9]|[12][0-9]|3[01])[/-](?:0[1-9]|1[0-2])[/-](?:\d{2,4})\b|'  # dd/mm/yyyy, dd-mm-yyyy
    r'\b(?:0[1-9]|[12][0-9]|3[01])[/-](?:0[1-9]|1[0-2])\b|'                    # dd/mm, dd-mm
    r'\b(?:0[1-9]|1[0-2])[/-]\d{4}\b|'                                           # mm/yyyy, mm-dd
    r'\b(?:0[1-9]|1[0-2])/\d{1,2}\b'                                             # mm/dd
    r'\b\d{1,2} (tháng )?\d{1,2}( năm \d{4})?\b'  # 06 tháng 12 năm 2015
)
    text = re.sub(pattern_date, '', text)

    tag_pattern = r'\b[A-Za-z]+\d+\b|\b\d+[A-Za-z]+\b|\b\d+[-_/][A-Za-z\d]+(?:[-_/][A-Za-z\d]+)*\b'

    preserved_tags = re.findall(tag_pattern, text)

    text = re.sub(tag_pattern, '<TAG>', text)

    pattern_long_numbers = r'\b\d{5,}\b'
    text = re.sub(pattern_long_numbers, '', text)

    pattern_isbn = r'\b\d{1,3}(?:-\d{1,5})+\b'
    text = re.sub(pattern_isbn, '', text)

    pattern_version = r'\bv?\d+(?:[.,]\d+)*\b'  #v1.2.345, 1.2.3.4
    text = re.sub(pattern_version, '', text)

    pattern_phone = r'\b(?:\+?\d{1,3}[ -]?)?(?:\d{2,4}[ -\.]?){2,3}\d{2,4}\b'
    text = re.sub(pattern_phone, '', text)

    pattern_ip = r'\b(?:\d{1,3}\.){3}\d{1,3}\b|\b(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}\b'
    text = re.sub(pattern_ip, '', text)

    pattern_percentage = r'\b\d+(?:,\d+)?%\b'
    text = re.sub(pattern_percentage, '', text)

    pattern_score = r'\b\d+\s*-\s*\d+\b'
    text = re.sub(pattern_score, '', text)

    pattern_time = r'\b\d{1,2}h(?:\d{2})?\b|\b\d{1,2}:\d{2}\b|\b\d{1,2}\+\d{1,2}\b'
    text = re.sub(pattern_time, '', text)

    text = re.sub(r'Ảnh: @ \w+\.?', '', text)

    for tag in preserved_tags:
        text = text.replace('<TAG>', tag, 1)

    return text

In [ ]:
dat = "A02 , 12/8 , 1.2.3 , 2004 , 12/06/2015 12/2015 4K, CIIB2426001 , 102/2024/NĐ-CP , 2 , 79/2024 , 12h , 20.000 ; 123456 ; 123.456,789"

In [ ]:
text = remove_special_num(dat)
print(text)

A02 , 12/8 ,  ,  ,   4K, CIIB2426001 , 102/2024/NĐ-CP ,  , 79/2024 , 12h ,  ;  ; 
